In [1]:
import requests 
import json
import pandas as pd 
from pandas import DataFrame
import geopandas as gpd
from matplotlib import artist
import matplotlib.pyplot as plt
import contextily as ctx
from pandas import DataFrame
from PIL import Image, ExifTags
from datetime import datetime


In [2]:
urlBase = 'http://localhost:8000/'
testpictures = 'C:/Smart/Support/bh_autoemail/autoemail/test-pictures'

<h1>Read exif Data from Picture</h1>

In [3]:
filename = testpictures + "/GP__0195.JPG"
print(filename)
img = Image.open(filename)
img_exif = img.getexif()
print(img_exif)

for key, val in img_exif.items():
    #print(key)
    if key in ExifTags.TAGS:
        print(f'{ExifTags.TAGS[key]}:{val}')
    else:
        print(f'{key}:{val}')

dtstr = img_exif.get(306)
dt = datetime.strptime(dtstr, '%Y:%m:%d %H:%M:%S')
print(dt)

C:/Smart/Support/bh_autoemail/autoemail/test-pictures/GP__0195.JPG
{296: 2, 282: 72.0, 34853: 686, 34665: 158, 270: 'DCIM\\100GOPRO\\GP__0195.JPG\x00\x00\x00\x00\x00', 271: 'GoPro\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 272: 'GoPro Max\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 305: 'H19.03.02.00.00\x00\x00\x00\x00\x00\x00\x00\x00', 274: 1, 306: '2022:08:26 16:30:44', 531: 1, 283: 72.0}
GPSInfo:686
ResolutionUnit:2
ExifOffset:158
ImageDescription:DCIM\100GOPRO\GP__0195.JPG     
Make:GoPro                        
Model:GoPro Max                    
Software:H19.03.02.00.00        
Orientation:1
DateTime:2022:08:26 16:30:44
YCbCrPositioning:1
XResolution:72.0
YResolution:72.0
2022-08-26 16:30:44


<h1>Find a matching patrol</h1>

In [4]:
session = requests.Session()
params = {}
#params["language_uuid"] = "2af26704-7dd9-4a77-bbd4-c1953800d4f8"
#params["timestamp"] = "2023-01-01T07:40:10"
params["start_date"] = "2023-01-01T00:00:00"
params["end_date"] = "2023-02-01T00:00:00"
#params["patrolleader_uuid"] = "0845746e-9ccb-4208-9b3c-fb16cc0ee991"
#params["polygon"] = 'MULTIPOINT (14.2461683333333 -19.477095,14.3404583333333 -19.4504783333333)'
#params["polygon"] = 'MULTIPOLYGON(((13.7284894201645 -19.3443099026193,13.717 -19.1337,13.6585 -18.9506,13.6596265556292 -18.9509276930682,13.7221 -18.9691,13.7542 -18.94,13.7626669151541 -18.9537424980453,13.9041 -19.1833,14.0513 -19.2743,13.9922 -19.5254,13.9743 -19.6093,13.9750000000019 -19.6666666696,13.9430555600018 -19.6277777796006,13.9166666700017 -19.604166669601,13.882776272314 -19.6006607637807,13.8763888900015 -19.5999999996011,13.8763892777546 -19.6000013731913,13.8760563387904 -19.5999673998276,13.8758104423102 -19.5999423015655,13.8758104244424 -19.5999422997424,13.8757555054052 -19.5999366962393,13.875755503989 -19.5999366940993,13.8752309318559 -19.5998831658502,13.8550000560976 -19.5390001607782,13.8550000000274 -19.5389999920401,13.7650000000112 -19.5079999916516,13.7610000000097 -19.428999992074,13.7300000000014 -19.3719999920918,13.7299590468362 -19.3712492890956,13.7284894201645 -19.3443099026193)))'
params["area_geometry_uuid"] = "4ebdaf23-61ff-483c-b51c-586cdf15ef54"
#params["limit"] = "1"
#params["employee_uuid"] = "0845746e-9ccb-4208-9b3c-fb16cc0ee991"
#params["limit"] = 10
r = session.get(urlBase + "patrol/find/", verify=False, params=params) 
#print(r.status_code)
j = json.loads(r.content)
#df = DataFrame.from_dict(j)
df = gpd.GeoDataFrame.from_features(j, crs=4326)
df
#print(json.dumps(j, indent=2))

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /patrol/find/?start_date=2023-01-01T00%3A00%3A00&end_date=2023-02-01T00%3A00%3A00&area_geometry_uuid=4ebdaf23-61ff-483c-b51c-586cdf15ef54 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001DDA2BF8090>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [15]:
df.explore(column='id', 
           #scheme='BoxPlot',
           #scheme='EqualInterval',
           #scheme='FisherJenks',
           scheme='Percentiles',
           legend=False
          )

In [4]:
#df[['patrol_start','patrol_end']].max()
df.groupby('patrol_leader_name').count()


NameError: name 'df' is not defined

In [6]:
df.groupby('patrol_leader_name')[['distance','duration(h)']].sum()

distance  duration(h)
patrol_leader_name                             
Benson Hiriua           240.090546    82.045279
Esau Matundu            113.571810    34.515001
Jackson Kavetu           44.148500    -1.330833
Kauhepere Musaso         12.688300     2.933889
Kaveisire Rutavi          0.000000     0.000000
Kavekaetua Tjauira       46.367100    11.268889
Mbaendeka Kangombe       20.497800     3.067500
Michael Ivanga Kapuike   18.264370     3.697500
Rinoveni Tjauira        178.992520    54.079167
Tjino Musaso             98.958300    18.017499
Uaroua Kaidue            17.345010     8.436389
Undari Hoveka            62.915090    27.318611
Unity Katjirumbu         14.951300     0.919722

In [7]:
df.track[1]

'MULTILINESTRING((13.9846083333333 -19.14587,13.9846083333333 -19.14587,13.98469 -19.14589,13.984005 -19.1455583333333,13.9846166666667 -19.144045,13.984555 -19.144345,13.985045 -19.1445433333333,13.9869266666667 -19.1448816666667,13.9907566666667 -19.1453233333333,13.9916483333333 -19.1437383333333,13.9922316666667 -19.139825,13.9921416666667 -19.1361566666667,13.9921016666667 -19.13518,13.991845 -19.12985,13.9907933333333 -19.1267016666667,13.9872916666667 -19.1249266666667,13.9865433333333 -19.1237083333333,13.9861283333333 -19.12347,13.9852683333333 -19.1222666666667,13.95317 -19.1210516666667,13.9530833333333 -19.1209983333333,13.9531316666667 -19.1209933333333,13.953125 -19.1208866666667,13.9532333333333 -19.1209816666667,13.958735 -19.130035,13.9601983333333 -19.1344683333333,13.9809016666667 -19.1414733333333,13.980675 -19.1419))'

<H1>Upload the file to SMART API</H1>

In [23]:
session = requests.Session()
params = {}
#params["language_uuid"] = "2af26704-7dd9-4a77-bbd4-c1953800d4f8"
#params["timestamp"] = "2023-01-01T07:40:10"
#params["start_date"] = "2023-01-01T00:00:00"
#params["end_date"] = "2023-02-01T00:00:00"
#params["patrolleader_uuid"] = "0845746e-9ccb-4208-9b3c-fb16cc0ee991"
#params["polygon"] = 'MULTIPOINT (14.2461683333333 -19.477095,14.3404583333333 -19.4504783333333)'
#params["polygon"] = 'MULTIPOLYGON(((13.7284894201645 -19.3443099026193,13.717 -19.1337,13.6585 -18.9506,13.6596265556292 -18.9509276930682,13.7221 -18.9691,13.7542 -18.94,13.7626669151541 -18.9537424980453,13.9041 -19.1833,14.0513 -19.2743,13.9922 -19.5254,13.9743 -19.6093,13.9750000000019 -19.6666666696,13.9430555600018 -19.6277777796006,13.9166666700017 -19.604166669601,13.882776272314 -19.6006607637807,13.8763888900015 -19.5999999996011,13.8763892777546 -19.6000013731913,13.8760563387904 -19.5999673998276,13.8758104423102 -19.5999423015655,13.8758104244424 -19.5999422997424,13.8757555054052 -19.5999366962393,13.875755503989 -19.5999366940993,13.8752309318559 -19.5998831658502,13.8550000560976 -19.5390001607782,13.8550000000274 -19.5389999920401,13.7650000000112 -19.5079999916516,13.7610000000097 -19.428999992074,13.7300000000014 -19.3719999920918,13.7299590468362 -19.3712492890956,13.7284894201645 -19.3443099026193)))'
#params["area_geometry_uuid"] = "4ebdaf23-61ff-483c-b51c-586cdf15ef54"
#params["limit"] = "1"
#params["employee_uuid"] = "0845746e-9ccb-4208-9b3c-fb16cc0ee991"
#params["limit"] = 10
params["patrol_uuid"] = "0eb78c4a-0757-4cca-a987-343d022e85c5"  # 
r = session.get(f"{urlBase}observation/patrolobservations/", verify=False, params=params) 
#print(r.status_code)
j = json.loads(r.content)
df = DataFrame.from_dict(j)
df


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [34]:
file = open(filename, "rb")
url = f"{urlBase}observation/create_observations/"
params = {}
params["patrol_uuid"] = "0eb78c4a-0757-4cca-a987-343d022e85c5"  # 
params["waypoint_uuid"] = "0fcbdf76-7498-4df0-90d0-0dd3db4b8169"
params["category_uuid"] = "ee72fc4d-c0b1-4dc1-9baf-a2b517ae9567"


test_resp = requests.post(url, files = {"file": file}, params=params)
if test_resp.ok:
    print(" File successfully uploaded ! ")
    print(test_resp.text)
else:
    print(" Please try again ! ")

 File successfully uploaded ! 
{"message":"Successfully uploaded GP__0195.JPG"}
